# Uploading more stuff

    Preparations: you know the drill by now

In [7]:
import pandas as pd
import psycopg2

In [8]:
con = psycopg2.connect(database="iSupply",
                        host="38.180.117.52",
                        user="postgres",
                        password="deerRun",
                        port="5432")
cursor = con.cursor()

cursor.close()
con.close()

    For this phase, we will need to read, so lets get that done

In [9]:
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432"); cursor = con.cursor()

cursor.execute("""select * from "Categories" c""")
out = cursor.fetchall(); print(out)

cursor.close(); con.close()

[(1, 'Agriculture'), (2, 'Agricultural Equipment'), (3, 'Agricultural Greenhouses'), (4, 'Aquaculture Equipment'), (5, 'Aquaculture Trap'), (6, 'Fishing Float'), (7, 'Fishing Net'), (8, 'Fishing Rope'), (9, 'Other Aquaculture Equipment'), (10, 'Ear Tag'), (11, 'Agricultural Growing Media'), (12, 'Peat'), (13, 'Agricultural Product Stock'), (14, 'Agricultural Waste'), (15, 'Animal Extract'), (16, 'Animal Feed'), (17, 'Animal Products'), (18, 'Eggs'), (19, 'Fowl & Livestock'), (20, 'Bamboo Raw Materials'), (21, 'Bean-Like Seeds'), (22, 'Cacao'), (23, 'Coffee Beans'), (24, 'Vanilla Beans'), (25, 'Beans'), (26, 'Broad Beans'), (27, 'Butter Beans'), (28, 'Chickpeas'), (29, 'Kidney Beans'), (30, 'Lentils'), (31, 'Lima Beans'), (32, 'Other Beans'), (33, 'Peas'), (34, 'Soybeans'), (35, 'Vigna Beans'), (36, 'Cigarettes'), (37, 'Cigars'), (38, 'Fresh Fruit'), (39, 'Fresh Apples'), (40, 'Fresh Apricots'), (41, 'Fresh Avocados'), (42, 'Fresh Bananas'), (43, 'Fresh Berries'), (44, 'Fresh Cherries')

    Now lets make this usable

In [10]:
df = pd.DataFrame(out, columns=['ser', 'cat']); print(df)

       ser                       cat
0        1               Agriculture
1        2    Agricultural Equipment
2        3  Agricultural Greenhouses
3        4     Aquaculture Equipment
4        5          Aquaculture Trap
...    ...                       ...
4147  4592            Truck Steering
4148  4593          Truck Suspension
4149  4594                Truck Tire
4150  4595        Truck Transmission
4151  4596               Truck Wheel

[4152 rows x 2 columns]


In [11]:
df.head(15)

,ser,cat
0,1,Agriculture
1,2,Agricultural Equipment
2,3,Agricultural Greenhouses
3,4,Aquaculture Equipment
4,5,Aquaculture Trap
5,6,Fishing Float
6,7,Fishing Net
7,8,Fishing Rope
8,9,Other Aquaculture Equipment
9,10,Ear Tag


    Now the next step is to take our tree from slimtree.xlsx, convert all pointers in the dataset from cat to ser (so the DB can eat them as int), & upload it

In [13]:
dfT = pd.read_excel("slimTree.xlsx")
dfT.head(15)

,category,subcategory
0,Agriculture,Agricultural Equipment
1,Agricultural Equipment,Agricultural Greenhouses
2,Agricultural Greenhouses,NaN
3,Agriculture,Agricultural Equipment
4,Agricultural Equipment,Aquaculture Equipment
5,Aquaculture Equipment,NaN
6,Agriculture,Agricultural Equipment
7,Agricultural Equipment,Aquaculture Equipment
8,Aquaculture Equipment,Aquaculture Trap
9,Agriculture,Agricultural Equipment


    Lets start by casting -1 to all leafs

In [14]:
dfT.fillna('-1', inplace=True); dfT.head(15)

,category,subcategory
0,Agriculture,Agricultural Equipment
1,Agricultural Equipment,Agricultural Greenhouses
2,Agricultural Greenhouses,-1
3,Agriculture,Agricultural Equipment
4,Agricultural Equipment,Aquaculture Equipment
5,Aquaculture Equipment,-1
6,Agriculture,Agricultural Equipment
7,Agricultural Equipment,Aquaculture Equipment
8,Aquaculture Equipment,Aquaculture Trap
9,Agriculture,Agricultural Equipment


    & then lets try replacing everything where needed

In [15]:
map_dic = df.set_index('cat')['ser'].to_dict()

dfInt = pd.DataFrame(dfT)

dfInt['category']    = dfInt['category']   .map(map_dic).fillna(-1).astype(int)
dfInt['subcategory'] = dfInt['subcategory'].map(map_dic).fillna(-1).astype(int)

dfInt.head(15)

,category,subcategory
0,1,2
1,2,3
2,3,-1
3,1,2
4,2,4
5,4,-1
6,1,2
7,2,4
8,4,5
9,1,2


    Cautious optimism time, lets investigate it

In [19]:
dfInt.iloc[0:10]

,category,subcategory
0,1,2
1,2,3
2,3,-1
3,1,2
4,2,4
5,4,-1
6,1,2
7,2,4
8,4,5
9,1,2


In [20]:
dfInt.iloc[100:110]

,category,subcategory
100,36,35
101,35,-1
102,1,37
103,37,35
104,35,-1
105,1,38
106,38,35
107,35,-1
108,1,38
109,38,39


In [21]:
dfInt.iloc[1000:1100]

,category,subcategory
1000,328,-1
1001,-1,-1
1002,152,328
1003,328,-1
1004,-1,-1
...,...,...
1095,-1,367
1096,367,368
1097,368,-1
1098,-1,367


In [23]:
dfInt.iloc[4500:4510]

,category,subcategory
4500,-1,1496
4501,1496,1500
4502,1500,-1
4503,-1,1496
4504,1496,1500
4505,1500,-1
4506,-1,1496
4507,1496,1500
4508,1500,-1
4509,-1,1496


In [26]:
print("The dataframe has ", dfInt.size, " entries")

The dataframe has  27564  entries


    Looking good, lets release the repeats

In [28]:
dfIntUnique = pd.DataFrame(dfInt.drop_duplicates())
dfIntUnique.head(15)

,category,subcategory
0,1,2
1,2,3
2,3,-1
4,2,4
5,4,-1
8,4,5
11,4,6
14,4,7
17,4,8
20,4,9


In [29]:
print("Now the dataframe has ", dfIntUnique.size, " entries")

Now the dataframe has  15400  entries


    Perfect, do I dare try to upload this? Lets check for issues first

In [33]:
def is_not_int(val):
    return not isinstance(val, int)

non_int_mask = dfIntUnique.map(is_not_int)
non_int_locations = dfIntUnique[non_int_mask]

print("Non ints are at:"); print(non_int_locations.dropna())


Non ints are at:
Empty DataFrame
Columns: [category, subcategory]
Index: []


    My favorite part was when they said ITS UPLOADING TIME & uploaded all over the place

In [49]:

def get_category_subcategory(df, index):
    parent = df.loc[index, "category"]                                                                                  # O( log(n) )
    child = df.loc[index, "subcategory"]
    return parent, child


def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; listOfAllErrors = []
    j = 0

    for i in range(start, end):                                                                                         # O(n)
        parent, child = get_category_subcategory(df, i)

        try:
            cursor.execute(
                """INSERT INTO "Category_Tree" (category_id, sub_category_id) VALUES (""" + parent + """, """ + child + """)""")#, (parent, child),)     # O(n)
        except psycopg2.errors.UniqueViolation as e:
            listOfAllErrors.append(i); listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
            listOfAllErrors.append(i)

        if j == 10:
            con.commit()
            j = 0  # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions), len(listOfAllErrors))
    return listOfCollisions, listOfAllErrors

In [50]:
listOfCollisions = []
listOfAllErrors = []

# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432" ); cursor = con.cursor()  # open

for i in range(2):  # first 200
    collisions, errors = process_categories(
        dfIntUnique, cursor, con, (100 * i), ((100 * i) + 100)
    )
    listOfCollisions += collisions
    listOfAllErrors += errors
    print("100 number ", (i + 1), " done")

cursor.close(); con.close()  # close

Error occurred: can only concatenate str (not "numpy.int32") to str
Error occurred: can only concatenate str (not "numpy.int32") to str
Error occurred: can only concatenate str (not "numpy.int32") to str


KeyError: 3

    Big error, lets snoop

In [51]:
con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432" ); cursor = con.cursor()  # open

parent = 12; child = -1
cursor.execute("""INSERT INTO "Category_Tree" (category_id, sub_category_id) VALUES (""" + str(parent) + """, """ + str(child) + """)""")

con.commit(); cursor.close(); con.close()  # close

    Don't you just love solving the problem first try, anyway lets re implement

In [54]:
def get_category_subcategory(df, index):
    parent = str(df.loc[index, "category"])                                                                                  # O( log(n) )
    child = str(df.loc[index, "subcategory"])
    return parent, child


def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; listOfAllErrors = []
    j = 0

    for i in range(start, end):                                                                                         # O(n)
        parent, child = get_category_subcategory(df, i)

        try:
            cursor.execute(
                """INSERT INTO "Category_Tree" (category_id, sub_category_id) VALUES (""" + parent + """, """ + child + """)""")    # O(n)
        except psycopg2.errors.UniqueViolation as e:
            listOfAllErrors.append(i); listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
            listOfAllErrors.append(i)

        if j == 10:
            con.commit()
            j = 0  # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions), len(listOfAllErrors))
    return listOfCollisions, listOfAllErrors

In [55]:
listOfCollisions = []
listOfAllErrors = []

# wipe the db from DBeaver

con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432" ); cursor = con.cursor()  # open

for i in range(2):  # first 200
    collisions, errors = process_categories(
        dfIntUnique, cursor, con, (100 * i), ((100 * i) + 100)
    )
    listOfCollisions += collisions
    listOfAllErrors += errors
    print("100 number ", (i + 1), " done")

cursor.close(); con.close()  # close

KeyError: 3

    Error remains, lets try again

In [58]:
def get_category_subcategory(df, index):
    parent = str(df.loc[index, "category"])                                                                                  # O( log(n) )
    child = str(df.loc[index, "subcategory"])
    return parent, child

parent, child = get_category_subcategory(dfIntUnique, 1); print(parent, child)

parent, child = get_category_subcategory(dfIntUnique, 2); print(parent, child)

parent, child = get_category_subcategory(dfIntUnique, 3); print(parent, child)


2 3
3 -1


KeyError: 3

    Theres the problem, lets try this

In [59]:
dfIntUnique.head(10)

,category,subcategory
0,1,2
1,2,3
2,3,-1
4,2,4
5,4,-1
8,4,5
11,4,6
14,4,7
17,4,8
20,4,9


    Problem confirmed, I shall fix

In [60]:
dfIntUnique.reset_index(drop=True, inplace=True); dfIntUnique.size

15400

In [61]:
dfIntUnique.head(10)

,category,subcategory
0,1,2
1,2,3
2,3,-1
3,2,4
4,4,-1
5,4,5
6,4,6
7,4,7
8,4,8
9,4,9


    Perfect, time to implement

In [62]:
def get_category_subcategory(df, index):
    parent = str(df.loc[index, "category"])                                                                                  # O( log(n) )
    child = str(df.loc[index, "subcategory"])
    return parent, child


def process_categories(df, cursor, con, start, end):
    listOfCollisions = []; listOfAllErrors = []
    j = 0

    for i in range(start, end):                                                                                         # O(n)
        parent, child = get_category_subcategory(df, i)

        try:
            cursor.execute(
                """INSERT INTO "Category_Tree" (category_id, sub_category_id) VALUES (""" + parent + """, """ + child + """)""")    # O(n)
        except psycopg2.errors.UniqueViolation as e:
            listOfAllErrors.append(i); listOfCollisions.append(i)
        except Exception as e:
            print(f"Error occurred: {e}")
            listOfAllErrors.append(i)

        if j == 10:
            con.commit()
            j = 0  # this time we will commit for each 10th entry
        j = j + 1

    con.commit()

    print(len(listOfCollisions), len(listOfAllErrors))
    return listOfCollisions, listOfAllErrors

In [63]:
listOfCollisions = []
listOfAllErrors = []

con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432" ); cursor = con.cursor()  # open

for i in range(2):  # first 200
    collisions, errors = process_categories(
        dfIntUnique, cursor, con, (100 * i), ((100 * i) + 100)
    )
    listOfCollisions += collisions
    listOfAllErrors += errors
    print("100 number ", (i + 1), " done")

cursor.close(); con.close()  # close

0 0
100 number  1  done
0 0
100 number  2  done


    Lets cross reference with DBeaver

![DBeaver pic](DBeaverShot.PNG "Screenshot from DBeaver")

In [64]:
dfIntUnique.head(20)

,category,subcategory
0,1,2
1,2,3
2,3,-1
3,2,4
4,4,-1
5,4,5
6,4,6
7,4,7
8,4,8
9,4,9


    Perfect, time for the big one

In [66]:
listOfCollisions = []
listOfAllErrors = []

# wipe db

con = psycopg2.connect(database="iSupply", host="38.180.117.52", user="postgres", password="deerRun", port="5432" ); cursor = con.cursor()  # open

for i in range(154):  # first 154 00, this will take under an hour
    collisions, errors = process_categories(
        dfIntUnique, cursor, con, (100 * i), ((100 * i) + 100)
    )
    listOfCollisions += collisions
    listOfAllErrors += errors
    print("100 number ", (i + 1), " done")

cursor.close(); con.close()  # close

0 0
100 number  1  done
0 0
100 number  2  done
0 0
100 number  3  done
0 0
100 number  4  done
0 0
100 number  5  done
0 0
100 number  6  done
0 0
100 number  7  done
0 0
100 number  8  done
0 0
100 number  9  done
0 0
100 number  10  done
0 0
100 number  11  done
0 0
100 number  12  done
0 0
100 number  13  done
0 0
100 number  14  done
0 0
100 number  15  done
0 0
100 number  16  done
0 0
100 number  17  done
0 0
100 number  18  done
0 0
100 number  19  done
0 0
100 number  20  done
0 0
100 number  21  done
0 0
100 number  22  done
0 0
100 number  23  done
0 0
100 number  24  done
0 0
100 number  25  done
0 0
100 number  26  done
0 0
100 number  27  done
0 0
100 number  28  done
0 0
100 number  29  done
0 0
100 number  30  done
0 0
100 number  31  done
0 0
100 number  32  done
0 0
100 number  33  done
0 0
100 number  34  done
0 0
100 number  35  done
0 0
100 number  36  done
0 0
100 number  37  done
0 0
100 number  38  done
0 0
100 number  39  done
0 0
100 number  40  done
0 0
100 n

KeyError: 7700

    bummer, only got 1/2 way

In [67]:
dfInt.iloc[7695:7705]

,category,subcategory
7695,-1,2535
7696,2535,2565
7697,2565,2568
7698,-1,2535
7699,2535,2565
7700,2565,2569
7701,-1,2535
7702,2535,2565
7703,2565,2570
7704,-1,2535


    After using the notebook for uploading in 2 halves:
    
    "it seems any possible number in front of 7700 will hurt the code, I will inspect in the notebook"
    
    it looks like any entry post 7700 has issues, lets see if this comes from the index

In [ ]:
# df2 = df.iloc[7700:15400].reset_index(drop=True)

# print(df2.head())